<a href="https://colab.research.google.com/github/Shashank-u803/AI-Lab/blob/main/AI_LAB_CIE_Resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Artificial Intelligence Lab CIE

Question: First Order Logic: Proof by Resolution technique Demonstration


Name: Shashank U

USN: 1BM23CS314

Date: 10/11/2025

In [ ]:
import re

# ------------------- REGEX & BASIC PARSING RULES -------------------
VAR_RE = re.compile(r'^[a-z]')
CONST_RE = re.compile(r'^[A-Z]')
PRED_RE = re.compile(r'^[A-Z].*\(')

def is_variable(token): return bool(VAR_RE.match(token))
def is_constant(token): return bool(CONST_RE.match(token)) and not bool(PRED_RE.match(token))
def is_predicate(text): return bool(PRED_RE.match(text))

def parse_literal(text):
    text = text.strip()
    neg = text.startswith(('¬', '~'))
    if neg:
        text = text[1:].strip()

    if is_predicate(text):
        pred = text[:text.index('(')].strip()
        args_str = text[text.index('(')+1:-1].strip()
        args = [a.strip() for a in args_str.split(',')] if args_str else []
    else:
        pred, args = text, []
    return {'neg': neg, 'pred': pred, 'args': args}

def clause_to_str(clause):
    if clause == []:
        return '⊥'
    return ' ∨ '.join([('¬' if lit['neg'] else '') +
                        (f"{lit['pred']}({', '.join(lit['args'])})" if lit['args'] else lit['pred'])
                        for lit in clause])

# ------------------- SAFE SPLITTING OF CLAUSES -------------------
def split_disjunction(line):
    parts, depth, current = [], 0, ""
    for ch in line:
        if ch == '(':
            depth += 1
        elif ch == ')':
            depth -= 1
        elif (ch == 'v' or ch == '∨') and depth == 0:
            parts.append(current.strip())
            current = ""
            continue
        current += ch
    if current.strip():
        parts.append(current.strip())
    return parts

# ------------------- STANDARDIZATION -------------------
def standardize_apart_clause(clause, idx):
    mapping, new_clause = {}, []
    for lit in clause:
        new_args = []
        for a in lit['args']:
            if is_variable(a):
                if a not in mapping:
                    mapping[a] = f"{a}_{idx}"
                new_args.append(mapping[a])
            else:
                new_args.append(a)
        new_clause.append({'neg': lit['neg'], 'pred': lit['pred'], 'args': new_args})
    return new_clause

# ------------------- UNIFICATION -------------------
def occurs_check(var, val, subs):
    if var == val: return True
    if is_variable(val) and val in subs:
        return occurs_check(var, subs[val], subs)
    return False

def apply_subs_token(tok, subs):
    while is_variable(tok) and tok in subs:
        tok = subs[tok]
    return tok

def apply_subs_literal(lit, subs):
    return {'neg': lit['neg'], 'pred': lit['pred'],
            'args': [apply_subs_token(a, subs) for a in lit['args']]}

def unify_tokens(x, y, subs):
    if x == y: return subs
    if is_variable(x):
        if x in subs: return unify_tokens(subs[x], y, subs)
        if occurs_check(x, y, subs): return None
        new = subs.copy(); new[x] = y; return new
    if is_variable(y): return unify_tokens(y, x, subs)
    return None

def unify_arg_lists(a_list, b_list):
    if len(a_list) != len(b_list): return None
    subs = {}
    for a, b in zip(a_list, b_list):
        a_applied, b_applied = apply_subs_token(a, subs), apply_subs_token(b, subs)
        subs = unify_tokens(a_applied, b_applied, subs)
        if subs is None: return None
    return subs

def is_tautology_clause(clause):
    seen = {}
    for lit in clause:
        key = (lit['pred'], tuple(lit['args']))
        if key in seen and seen[key] != lit['neg']:
            return True
        seen[key] = lit['neg']
    return False

# ------------------- NODE CLASS -------------------
class Node:
    def __init__(self, clause, parents=None, label=None, info=None):
        self.clause = clause
        self.parents = parents if parents else []
        self.label = label
        self.info = info  # explanation for how derived

# ------------------- RESOLUTION -------------------
def resolve_pair(c1, c2):
    resolvents = []
    for i, l1 in enumerate(c1):
        for j, l2 in enumerate(c2):
            if l1['pred'] == l2['pred'] and l1['neg'] != l2['neg']:
                subs = unify_arg_lists(l1['args'], l2['args']) if (l1['args'] or l2['args']) else {}
                if subs is None: continue

                new_clause = [apply_subs_literal(l, subs) for k,l in enumerate(c1) if k!=i] + \
                             [apply_subs_literal(l, subs) for k,l in enumerate(c2) if k!=j]

                uniq = []
                for lit in new_clause:
                    if not any(lit['neg']==u['neg'] and lit['pred']==u['pred'] and lit['args']==u['args'] for u in uniq):
                        uniq.append(lit)
                if is_tautology_clause(uniq): continue

                desc = (f"Resolved on {clause_to_str([l1])} and {clause_to_str([l2])}\n"
                        f"    Substitution: {subs}\n"
                        f"    Resulting clause: {clause_to_str(uniq) if uniq else '⊥'}")
                resolvents.append((uniq, subs, (i, j, desc)))
    return resolvents

# ------------------- MAIN RESOLUTION PROCEDURE -------------------
def resolution_with_tree(initial_clauses, goal_clause):
    clauses_nodes = []
    for idx, c in enumerate(initial_clauses):
        std = standardize_apart_clause(c, idx)
        clauses_nodes.append(Node(std, label=f"C{idx}"))

    neg_goal = [{'neg': not l['neg'], 'pred': l['pred'], 'args': l['args'][:]} for l in goal_clause]
    neg_goal_std = standardize_apart_clause(neg_goal, len(clauses_nodes))
    goal_node = Node(neg_goal_std, label="¬Goal")
    clauses_nodes.append(goal_node)

    print("\n==== Knowledge Base Clauses ====")
    for n in clauses_nodes:
        print(f"{n.label}: {clause_to_str(n.clause)}")

    seen = {clause_to_str(n.clause): i for i, n in enumerate(clauses_nodes)}
    idx_counter = len(clauses_nodes)

    iteration = 1
    while True:
        new_found = False
        print(f"\n---- Iteration {iteration} ----")
        for i in range(len(clauses_nodes)):
            for j in range(i + 1, len(clauses_nodes)):
                c1, c2 = clauses_nodes[i], clauses_nodes[j]
                resolvents = resolve_pair(c1.clause, c2.clause)
                for resolvent, subs, (_, _, desc) in resolvents:
                    s = clause_to_str(resolvent)
                    if s in seen: continue
                    new_node = Node(resolvent, parents=[i, j],
                                    label=f"R{idx_counter}", info=desc)
                    clauses_nodes.append(new_node)
                    seen[s] = idx_counter
                    print(f"{new_node.label}: {desc}\n    From {c1.label}, {c2.label}")
                    if resolvent == []:
                        print("\n*** Empty clause (⊥) derived! ***")
                        return clauses_nodes, idx_counter
                    idx_counter += 1
                    new_found = True
        if not new_found:
            break
        iteration += 1
    return clauses_nodes, None

# ------------------- PRINT TOP-DOWN TREE -------------------
def print_top_down_tree(nodes, root_idx):
    print("\n==== TOP-DOWN DERIVATION TREE ====")
    def recurse(idx, indent=""):
        node = nodes[idx]
        print(f"{indent}{node.label}: {clause_to_str(node.clause)}")
        if node.info:
            print(f"{indent}  {node.info.replace(chr(10), chr(10)+indent+'  ')}")
        for p in node.parents:
            recurse(p, indent + "    ")
    recurse(root_idx)

# ------------------- RUNNER -------------------
if __name__ == "__main__":
    print("="*70)
    print("FIRST-ORDER LOGIC RESOLUTION SYSTEM (Detailed Top-Down)")
    print("="*70)
    print("Enter CNF clauses (one per line). End with a blank line:")

    raw = []
    while True:
        line = input().strip()
        if not line:
            break
        raw.append(line)

    clauses = [[parse_literal(tok.strip()) for tok in split_disjunction(line)] for line in raw]

    goal_line = input("\nEnter GOAL clause (single literal form): ").strip()
    goal_clause = [parse_literal(goal_line)]

    nodes, root_idx = resolution_with_tree(clauses, goal_clause)
    if root_idx is None:
        print("\nNo empty clause could be derived — goal NOT entailed by KB.")
    else:
        print_top_down_tree(nodes, root_idx)
        print("\nResolution complete — contradiction found (⊥).")


FIRST-ORDER LOGIC RESOLUTION SYSTEM (Detailed Top-Down)
Enter CNF clauses (one per line). End with a blank line:
~Easy(course) v Likes(Steve,course)
~Science(course) v Hard(course)
~Compscience(course) v Easy(course)
Compscience(Ai)


Enter GOAL clause (single literal form): Likes(Steve,Ai)

==== Knowledge Base Clauses ====
C0: ¬Easy(course_0) ∨ Likes(Steve, course_0)
C1: ¬Science(course_1) ∨ Hard(course_1)
C2: ¬Compscience(course_2) ∨ Easy(course_2)
C3: Compscience(Ai)
¬Goal: ¬Likes(Steve, Ai)

---- Iteration 1 ----
R5: Resolved on ¬Easy(course_0) and Easy(course_2)
    Substitution: {'course_0': 'course_2'}
    Resulting clause: Likes(Steve, course_2) ∨ ¬Compscience(course_2)
    From C0, C2
R6: Resolved on Likes(Steve, course_0) and ¬Likes(Steve, Ai)
    Substitution: {'course_0': 'Ai'}
    Resulting clause: ¬Easy(Ai)
    From C0, ¬Goal
R7: Resolved on ¬Compscience(course_2) and Compscience(Ai)
    Substitution: {'course_2': 'Ai'}
    Resulting clause: Easy(Ai)
    From C2, C3
R8: R